# V. Landmark-based Localization and mapping. 
## Section A. Localization with error-state Kalman filter on manifold
Based on the [paper](https://arxiv.org/abs/1812.01537) by Sola and [manifpy](https://artivis.github.io/manif/python/) examples. 
### This is hugely based on [se2_localization.py](examples/se2_localization.py), which we have not written ourselves.

In [ ]:
from manifpy import SE2, SE2Tangent
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv

Will not explain the code, since it is already well explained in the paper and the manifpy examples. We added some visualization to the code to help understand the results.

In [ ]:

NUMBER_OF_LMKS_TO_MEASURE = 3

X_simulation = SE2.Identity()
X = SE2.Identity()
X_unfiltered = SE2.Identity()
P = np.zeros((3, 3))

u_nom = np.array([0.1, 0.0, 0.05])
u_sigmas = np.array([0.1, 0.1, 0.1])
U = np.diagflat(np.square(u_sigmas))

J_x = np.zeros((3, 3))
J_u = np.zeros((3, 3))

landmarks = np.array([
    [2.0,  0.0],
    [2.0,  1.0],
    [2.0, -1.0]
])

measurements = [np.array([0, 0])] * NUMBER_OF_LMKS_TO_MEASURE

y_sigmas = np.array([0.01, 0.01])
R = np.diagflat(np.square(y_sigmas))

J_xi_x = np.zeros((3, 3))
J_e_xi = np.zeros((2, 3))

X_sim = np.zeros((10, 3))
X_est = np.zeros((10, 3))
X_unf = np.zeros((10, 3))

In [ ]:
for t in range(10):
        # I. Simulation

        # simulate noise
        u_noise = u_sigmas * np.random.rand(SE2.DoF)        # control noise
        u_noisy = u_nom + u_noise                           # noisy control

        u_simu = SE2Tangent(u_nom)
        u_est = SE2Tangent(u_noisy)
        u_unfilt = SE2Tangent(u_noisy)

        # first we move
        X_simulation = X_simulation + u_simu                # overloaded X.rplus(u) = X * exp(u)

        # then we measure all landmarks
        for i in range(NUMBER_OF_LMKS_TO_MEASURE):
            b = landmarks[i]                                # lmk coordinates in world frame

            # simulate noise
            y_noise = y_sigmas * np.random.rand(SE2.Dim)    # measurement noise

            y = X_simulation.inverse().act(b)               # landmark measurement, before adding noise

            y = y + y_noise                                 # landmark measurement, noisy
            measurements[i] = y                             # store for the estimator just below

        # II. Estimation

        # First we move

        X = X.plus(u_est, J_x, J_u)                         # X * exp(u), with Jacobians

        P = J_x @ P @ J_x.transpose() + J_u @ U @ J_u.transpose()

        # Then we correct using the measurements of each lmk
        for i in range(NUMBER_OF_LMKS_TO_MEASURE):
            # landmark
            b = landmarks[i]                                # lmk coordinates in world frame

            # measurement
            y = measurements[i]                             # lmk measurement, noisy

            # expectation
            e = X.inverse(J_xi_x).act(b, J_e_xi)            # note: e = R.tr * ( b - t ), for X = (R,t).
            H = J_e_xi @ J_xi_x                             # Jacobian of the measurements wrt the robot pose. note: H = J_e_x = J_e_xi * J_xi_x
            E = H @ P @ H.transpose()

            # innovation
            z = y - e
            Z = E + R

            # Kalman gain
            K = P @ H.transpose() @ inv(Z)                  # K = P * H.tr * ( H * P * H.tr + R).inv

            # Correction step
            dx = K @ z                                      # dx is in the tangent space at X

            # Update
            X = X + SE2Tangent(dx)                          # overloaded X.rplus(dx) = X * exp(dx)
            P = P - K @ Z @ K.transpose()

        # III. Unfiltered

        # move also an unfiltered version for comparison purposes
        X_unfiltered = X_unfiltered + u_unfilt

        # IV. Results

        # Logging
        X_sim[t, :] =  np.array([X_simulation.x(), X_simulation.y(), X_simulation.angle()])
        X_est[t, :] =  np.array([X.x(), X.y(), X.angle()])
        X_unf[t, :] =  np.array([X_unfiltered.x(), X_unfiltered.y(), X_unfiltered.angle()])
        # END logging


In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.title('W. landmarks')
plt.scatter(X_sim[:, 0], X_sim[:, 1], color='k', label='simulated')
plt.scatter(X_est[:, 0], X_est[:, 1], color='r', label='estimated')
plt.scatter(X_unf[:, 0], X_unf[:, 1], color='b', label='unfiltered')
plt.scatter(landmarks[:, 0], landmarks[:, 1], color='y', label='Landmarks')
plt.legend()

plt.subplot(2, 2, 2)
plt.title('No landmarks')
plt.scatter(X_sim[:, 0], X_sim[:, 1], color='k', label='simulated')
plt.scatter(X_est[:, 0], X_est[:, 1], color='r', label='estimated')
plt.scatter(X_unf[:, 0], X_unf[:, 1], color='b', label='unfiltered')
plt.legend()

plt.subplot(2, 2, 3)
plt.title('Simulated vs estimated')
plt.scatter(X_sim[:, 0], X_sim[:, 1], color='k', label='simulated')
plt.scatter(X_est[:, 0], X_est[:, 1], color='r', label='estimated')
plt.legend()

plt.subplot(2, 2, 4)
plt.title('Directions')
plt.quiver(X_sim[:, 0], X_sim[:, 1], np.cos(X_sim[:, 2]), np.sin(X_sim[:, 2]), color='k', label='simulated')
plt.quiver(X_est[:, 0], X_est[:, 1], np.cos(X_est[:, 2]), np.sin(X_est[:, 2]), color='r', label='estimated')
plt.legend()
plt.suptitle('Localization')
plt.savefig("examples/localization_output.png")

plt.show()